In [1]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

## Criar e gerenciar um repositório de vetores (vector store) onde arquivos PDF podem ser armazenados e processados para recuperação de informações de forma eficiente.

In [2]:
# Cria um repositório de vetores chamado "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Financial Statements")

# Lista os caminhos dos arquivos PDF a serem carregados
file_paths = ["arquivos/divulgacao_resultado_ambev_4T23.pdf"]  # Pode ser mais de um

# Abre os arquivos em modo de leitura binária e armazena os streams em uma lista
file_streams = [open(path, "rb") for path in file_paths]

# Utiliza-se função da API para carregar os arquivos no repositório de vetores e monitorar o status do upload até que a conclusão.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# Imprime o status do batch de arquivos para verificar se a operação foi concluída com sucesso
print(file_batch.status)

# Imprime a contagem de arquivos processados no batch
print(file_batch.file_counts)


completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


# Cria o assistant

In [14]:
assistant = client.beta.assistants.create(
  name="Financial Analyst Assistant",
  instructions="Você é um analista de demonstralções financeiras da Ambev. Você tem acesso a demontração de resultado do \
                4º trimestre de 2023. Baseado apenas no documento que você tem acesso, responda as perguntas do usuário.",
  model="gpt-3.5-turbo-0125",  # gpt-4o
  tools=[{"type": "file_search"}],
  #tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

### Se o seu objetivo é apenas processar e armazenar os arquivos no repositório de vetores para consultas ou análises, o primeiro upload é suficiente. Se você precisar que o assistente utilize o arquivo diretamente, então o segundo upload pode ser necessário.

In [15]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("arquivos/divulgacao_resultado_ambev_4T23.pdf", "rb"), purpose="assistants"
)

# Cria a thread

In [16]:
# Criar um tópico e anexar o arquivo à mensagem
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Qual o volume de cerja vendido no Brasil?",
       # Anexar o novo arquivo à mensagem
        "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
      
    }
  ]
)
 
# O tópico agora tem um repositório de vetores com esse arquivo em seus recursos de ferramentas.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_y3whrEIrGPcJA2ngqk1O5Q4Y'])


## Roda a thread no assistant

In [17]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='O nome do usuário é Adriano.'
)

## Aguarda a thread rodar

In [18]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

print(run.status)

completed


## Verifica a resposta

In [19]:
if run.status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(messages)
else:
    print('Erro', run.status)

SyncCursorPage[Message](data=[Message(id='msg_KDzNtRzngnARRd6mltqaLRva', assistant_id='asst_7vMxL5vRXuOMcWwAJM4BekHJ', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=117, file_citation=FileCitation(file_id='file-lm79bEdjQQHMJmbAIPHJMO8u', quote=None), start_index=105, text='【4:5†source】', type='file_citation'), FileCitationAnnotation(end_index=129, file_citation=FileCitation(file_id='file-lm79bEdjQQHMJmbAIPHJMO8u', quote=None), start_index=117, text='【4:6†source】', type='file_citation'), FileCitationAnnotation(end_index=226, file_citation=FileCitation(file_id='file-lm79bEdjQQHMJmbAIPHJMO8u', quote=None), start_index=214, text='【4:5†source】', type='file_citation'), FileCitationAnnotation(end_index=238, file_citation=FileCitation(file_id='file-lm79bEdjQQHMJmbAIPHJMO8u', quote=None), start_index=226, text='【4:6†source】', type='file_citation'), FileCitationAnnotation(end_index=435, file_citation=FileCitation(file_id='fil

### Resposta

In [20]:
print(messages.data[0].content[0].text.value)

O volume de cerveja vendido no Brasil em 2023 pela Ambev foi de aproximadamente 93 milhões de hectolitros【4:5†source】【4:6†source】. Isso representa uma pequena diminuição de 1,0% em relação ao volume vendido em 2022【4:5†source】【4:6†source】.

Para o quarto trimestre de 2023 especificamente, o volume foi de aproximadamente 26,3 milhões de hectolitros, uma diminuição de 1,1% quando comparado ao mesmo período do ano anterior【4:0†source】【4:1†source】.


## Analisando os passos do modelo

In [10]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [11]:
for step in run_steps.data[::-1]:
    print(" ")
    print('======= Step >', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            if tool_call.type == 'file_search':
                print(tool_call)
            else:
                print('```')
                print(tool_call.code_interpreter.input)
                print('```')
                if tool_call.code_interpreter.outputs[0].type == 'logs':
                    print('Result')
                    print(tool_call.code_interpreter.outputs[0].logs)
                    
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == 'text':
            message = client.beta.threads.messages.retrieve(
                thread_id=thread.id,
                message_id=step.step_details.message_creation.message_id
            )
            print(message.content[0].text.value)

        if message.content[0].type == 'image_file':
            message = client.beta.threads.messages.retrieve(
                thread_id=thread.id,
                message_id=step.step_details.message_creation.message_id
            )
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)

            with open(f'arquivos/{file_id}.png', 'wb') as file:
                file.write(image_data.read())

            import matplotlib.pyplot as plt
            import matplotlib.image as mpimg

            img = mpimg.imread(f'arquivos/{file_id}.png')
            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(img)
            plt.show()
        

 
======= Step > tool_calls
FileSearchToolCall(id='call_ydfJqNLwq63uD8edlgsQngrn', file_search={}, type='file_search')
 
======= Step > message_creation
No Brasil, foram vendidos aproximadamente 93.111,6 mil hectolitros de cerveja, com uma variação de -1,0% em relação ao período anterior【4:2†source】.


In [12]:
step.step_details

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_Mio6iorUuZ8TBdv3M1uGiny2'), type='message_creation')

In [13]:
tool_call

FileSearchToolCall(id='call_ydfJqNLwq63uD8edlgsQngrn', file_search={}, type='file_search')